In [8]:
import operator
import os
import random
from typing import Annotated, Sequence, TypedDict

from dotenv import load_dotenv
from IPython.display import Image
from langchain.agents import (AgentExecutor, AgentType,
                              create_openai_functions_agent, initialize_agent)
from langchain_core.messages import (BaseMessage, HumanMessage, SystemMessage,
                                     ToolMessage)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.graph import END, MessageGraph, StateGraph
from pptx import Presentation
from IPython.display import Image

load_dotenv()

llm = ChatOpenAI(model="gpt-4o-mini")

In [6]:
def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system": system_prompt),
            MessagesPlaceholder()
        ]
    ) 

SyntaxError: invalid syntax (882076681.py, line 4)